In [10]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━

In [2]:
# 로드
import time
import datetime
import os
import copy
import cv2
import random
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from PIL import Image
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm
import torch.optim as optim

In [4]:
model_name = 'efficientnet-b6'
num_classes = 4  # 여기선 고정값 아웃풋 0 1 2 3 // 4가지

In [21]:
#웹페이지에서 이미지를 업로드 받은 후 연산시간을 줄이기 위해 미리 로드
#PATH
#checkpoint_model2.pt
PATH2 = '/content/drive/MyDrive/capstone/scalp_weights/'+'checkpoint_model2.pt'  # 모델2 피지과다
PATH3 = '/content/drive/MyDrive/capstone/scalp_weights/'+'checkpoint_model3.pt'  # 모델3 모낭사이홍반
PATH6 = '/content/drive/MyDrive/capstone/scalp_weights/'+'checkpoint_model6.pt'  # 모델6 탈모

#cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#모델 로드

model2 = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)  # 모델 다운로드
model3 = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)
model6 = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)

# 저장된 state_dict 로드

checkpoint2 = torch.load(PATH2)
checkpoint3 = torch.load(PATH3)
checkpoint6 = torch.load(PATH6)

# state_dict를 각각의 모델에 로드
model2.load_state_dict(checkpoint2['model_state_dict'])
model3.load_state_dict(checkpoint3['model_state_dict'])
model6.load_state_dict(checkpoint6['model_state_dict'])

#모델을 평가모드로 전환   # 평가모드와 학습모드의 layer 구성이 다르다
model2.eval()
model3.eval()
model6.eval()

Loaded pretrained weights for efficientnet-b6
Loaded pretrained weights for efficientnet-b6
Loaded pretrained weights for efficientnet-b6


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 56, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        56, 56, kernel_size=(3, 3), stride=[1, 1], groups=56, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        56, 14, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        14, 56, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        56, 32, kernel_size=(1, 1), stride=(1, 1), bias=False
  

In [32]:
#전처리 : 트랜스폼 규칙 선언 # validation set 의 트랜스폼 규칙과 동일
transforms_test = transforms.Compose([  transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BOX),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])    ])
#root 경로 폴더 속 jpg를 전처리, 텐서화 # /content/drive/MyDrive/capstone/
testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/capstone/test_data',
                    transform = transforms_test)
#앞서 전처리한 testset 을 DataLoader를 통해 네트워크에 올리기
testloader = DataLoader(testset, batch_size=2, shuffle=False, num_workers=0)
    #데이터 로더는 데이터의 대량 가져오기 또는 내보내기를 위한 클라이언트 응용 프로그램

In [36]:
#for data, target in testloader: 에서 data는 데이터의 특징  target은 데이터의 정답값
all_diagnoses = []
with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
  for data, target in testloader:
    data, target  = data.to(device), target.to(device)
    output2 = model2(data) # model에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
    output3 = model3(data)
    output6 = model6(data)
    #predict # # 0~3값만 뽑기
    m2p = output2.argmax(dim=1, keepdim=True)[0][0].tolist()
    m3p = output3.argmax(dim=1, keepdim=True)[0][0].tolist()
    m6p = output6.argmax(dim=1, keepdim=True)[0][0].tolist()

   # 두피 유형 진단
    diagnosis = []
    if m2p == 0 and m3p == 0 and m6p == 0:
        diagnosis.append('정상입니다.')
    if m2p != 0:
        if m2p == 1:
            diagnosis.append('지성 두피 (경증)')
        elif m2p == 2:
            diagnosis.append('지성 두피 (중증)')
        elif m2p == 3:
            diagnosis.append('지성 두피 (중증도)')
    if m3p != 0:
        if m3p == 1:
            diagnosis.append('민감성 두피 (경증)')
        elif m3p == 2:
            diagnosis.append('민감성 두피 (중증)')
        elif m3p == 3:
            diagnosis.append('민감성 두피 (중증도)')
    if m2p != 0 and m3p != 0:
        if m2p == 1 and m3p == 1:
            diagnosis.append('지루성 두피 (경증)')
        elif m2p == 2 and m3p == 2:
            diagnosis.append('지루성 두피 (중증)')
        elif m2p == 3 and m3p == 3:
            diagnosis.append('지루성 두피 (중증도)')
    if m6p != 0:
        if m6p == 1:
            diagnosis.append('탈모 (경증)')
        elif m6p == 2:
            diagnosis.append('탈모 (중증)')
        elif m6p == 3:
            diagnosis.append('탈모 (중증도)')

    # 진단 결과 저장
    all_diagnoses.append(", ".join(diagnosis))

    #print(f"Model2 prediction: {m2p}, Model3 prediction: {m3p}, Model6 prediction: {m6p}")

In [37]:
# 진단 결과 출력
for idx, diagnosis in enumerate(all_diagnoses):
    print(f"Image {idx + 1}: {diagnosis}")

Image 1: 지성 두피 (경증), 민감성 두피 (중증도), 탈모 (중증도)
Image 2: 탈모 (중증)
Image 3: 탈모 (경증)
Image 4: 지성 두피 (중증), 민감성 두피 (중증), 지루성 두피 (중증), 탈모 (경증)
